# Get weights from Fusion

In [1]:
cd /project2/mstephens/causalTWAS/fusion_weights

In [1]:
wget https://data.broadinstitute.org/alkesgroup/FUSION/WGT/YFS.BLOOD.RNAARR.tar.bz2

--2019-10-29 16:14:53--  https://data.broadinstitute.org/alkesgroup/FUSION/WGT/YFS.BLOOD.RNAARR.tar.bz2
Resolving data.broadinstitute.org (data.broadinstitute.org)... 69.173.92.29
Connecting to data.broadinstitute.org (data.broadinstitute.org)|69.173.92.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69800449 (67M) [application/x-bzip2]
Saving to: ‘YFS.BLOOD.RNAARR.tar.bz2’

100%[======================================>] 69,800,449  3.46MB/s   in 14s    

2019-10-29 16:15:08 (4.75 MB/s) - ‘YFS.BLOOD.RNAARR.tar.bz2’ saved [69800449/69800449]



In [3]:
tar xjf YFS.BLOOD.RNAARR.tar.bz2

In [2]:
ls

YFS.BLOOD.RNAARR       YFS.BLOOD.RNAARR.pos	 YFS.BLOOD.RNAARR.profile.err
YFS.BLOOD.RNAARR.list  YFS.BLOOD.RNAARR.profile  YFS.BLOOD.RNAARR.tar.bz2


* download all GTEx weights (v7) from Fusion

In [3]:
cd /project2/mstephens/causalTWAS/fusion_weights

In [4]:
wget http://gusevlab.org/projects/fusion/weights/GTEX7.txt

--2020-05-10 21:25:22--  http://gusevlab.org/projects/fusion/weights/GTEX7.txt
Resolving gusevlab.org (gusevlab.org)... 52.216.104.170
Connecting to gusevlab.org (gusevlab.org)|52.216.104.170|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3827 (3.7K) [text/plain]
Saving to: ‘GTEX7.txt’

100%[======================================>] 3,827       --.-K/s   in 0s      

2020-05-10 21:25:22 (215 MB/s) - ‘GTEX7.txt’ saved [3827/3827]



In [ ]:
wget -E -H -k -K -p -e robots=off -i ./GTEX7.txt

## Create multiple copies with different names

only keep chr17-22 genes, make 8 copies and assign names with tag B1-B8

In [4]:
cd /project2/mstephens/causalTWAS/fusion_weights
mkdir Adipose_Subcutaneous_B
sed 's\Adipose_Subcutaneous\Adipose_Subcutaneous_B\g' Adipose_Subcutaneous.pos > Adipose_Subcutaneous_B.pos

mkdir: cannot create directory ‘Adipose_Subcutaneous_B’: File exists


In [ ]:
# R code
setwd("/home/simingz/causalTWAS/fusion_weights")
posf0 <- "Adipose_Subcutaneous.pos"
posf = "Adipose_Subcutaneous_B.pos"
pos0 = read.table(posf0, header = T, stringsAsFactors = F)
pos <- read.table(posf, header = T, stringsAsFactors = F)

pos0 <- pos0[pos0$CHR <=22 & pos0$CHR >=17,]
pos <- pos[pos$CHR <=22 & pos$CHR >=17,]

renamesnps <- function(row, tag) {
  f0 <- row[1]
  f <- row[2]
  load(f0)
  rownames(wgt.matrix) <- paste0(rownames(wgt.matrix), tag)
  snps[,1] <- paste0(snps[,1], tag)
  snps[,2] <- paste0(snps[,2], tag)
  save(wgt.matrix, snps, cv.performance, hsq, hsq.pv, N.tot, file = f)
}


outdfall <- NULL
for (i in 1:6) {
  tag <- paste0("-B", i)
  outdf <- pos
  outdf$WGT <- gsub(".wgt", paste0(tag, ".wgt"), outdf$WGT)
  outdf$ID <- paste0(outdf$ID, tag)
  outdf$CHR <- paste0(outdf$CHR, tag)
  rd <- cbind(pos0$WGT, outdf$WGT)
  apply(rd, 1, renamesnps, tag = tag)
  outdfall <- rbind(outdfall, outdf)
}

write.table(outdfall , file= posf, row.names=F, col.names=T, sep="\t", quote = F)

# Fusion weights from Xiaotong

In [4]:
cp /project2/xinhe/xsun/former_proj/methylation_train/WEIGHTS/*.* /project2/mstephens/causalTWAS/fusion_weights

In [ ]:
cp /project2/xinhe/xsun/former_proj/train_weights_H3K27ac/WEIGHTS /project2/mstephens/causalTWAS/fusion_weights 

# Process WTCCC data

WTCCC data obtained from Peter (`bd.RData`). We will prepare genotype file in BIMBAM format and relatedness matrix file.

In [3]:
# sinteractive --time=1-12:00:00 --partition=mstephens --mem=20G
module load R/3.5.1

In [15]:
cd /project2/mstephens/causalTWAS/causal-TWAS/code

In [33]:
Rscript run_WTCCC_data_process.R

In [17]:
ls /project2/mstephens/causalTWAS/WTCCC

bd.geno.txt.gz	bd.pheno.txt  bd.QC.RData  bd.RData


# Gene annotation data

In [11]:
mkdir /project2/mstephens/causalTWAS/annotation

In [ ]:
cd /project2/mstephens/causalTWAS/annotation; wget http://hgdownload.soe.ucsc.edu/goldenPath/hg17/database/refGene.txt.gz

In [22]:
gunzip refGene.txt.gz

In [ ]:
Rscript /project2/mstephens/causalTWAS/causal-TWAS/code/gene_annotation.R refGene.txt 500000

In [24]:
ls

refGene_processed_5e+05.txt  refGene.txt


# UKbiobank data

## About CRI
Intructions to use CRI: https://github.com/stephenslab/gardner

* interactive job  
`qsub -I -l walltime=05:00:00 -l nodes=1:ppn=12 -l mem=32gb`
* submit a job  
`qsub ~/script/run_serverjob_template.sh`

## Data storage location
on CRI.
phenotype data: `/gpfs/data/xhe-lab/uk-biobank/`.  
genotype data:  `/gpfs/data/pierce-lab/uk-biobank-genotypes`.

## Prepare genotype file for chromosome 22, 20000 samples.

plink filtering instructions: https://www.cog-genomics.org/plink/2.0/filter

In [ ]:
# on CRI, the interactive node is very unstable and eventually killed due to memory isues
# prepare kept samples, take 20,000
cd /home/szhao1/uk-biobank/uk-biobank-genotypes; 
head -n 20002 ukb17346_imp_chr17_v3_s487378.sample | awk '{print $1"\t"$2}' > /home/szhao1/causal-TWAS/data/ukbiobank_samples20000.txt
# then manually remove the first two rows.
cd /home/szhao1/causal-TWAS/data; qsub qsub_prep_ukbiobank_n20000_chr22_genotype.sh

In [ ]:
# use sftp to download file to RCC (very fast)
cd /home/simingz/causalTWAS/ukbiobank
sftp szhao1@cri-syncmon.cri.uchicago.edu
cd /gpfs/data/pierce-lab/uk-biobank-genotypes
get ukb_imp_chr22_v3.bgen.*
get head -n 20002 ukb17346_imp_chr17_v3_s487378.sample

In [ ]:
head -n 20002 ukb17346_imp_chr17_v3_s487378.sample |  awk '{print $1"\t"$2}'> ukb17346_imp_chr17_v3_s20000.sample
# manually remove first two rows

**Process .bgen file using plink.**  
The original chr22 bgen file contains 1251k variants. After applied the maf (minor allele frequency) 5% cut off, 100k remains. then after applied missing rate 5% cut off (default is 10%), 80k remained.

Originall tried `--make-bed`, this resulted in missing genotype, so switched to `--make-pgen`.

> Some notes from Chirstopher Chang (author of plink)
1. You generally want to use PLINK 2.0 instead of 1.9 when working with .bgen files.  PLINK 1.9 is unable to track any of the genotype probability information in the .bgen; it rounds numbers to the nearest integer and replaces those too far from an integer with missing calls.
2. If you want to generate a .bgen file with only the variants you want, replace "--out rs6857.bgen" with "--export bgen-1.1 --out rs6857".  --out just specifies an output filename prefix; including ".bgen" in the --out parameter does not tell PLINK 2.0 to export a .bgen file.
3. This will still be a lossy process; PLINK 2.0 keeps track of dosages, but not genotype probabilities.  When sample x has P(AA) = 0.8, P(AC) = 0.2, and P(CC) = 0 and sample y has P(AA) = 0.85, P(AC) = 0.1, and P(CC) = 0.05, PLINK 2.0 sees both samples as dosage(A) = 1.8, dosage(C) = 0.2, and it will make both entries look like sample x when asked to export a .bgen.  Sometimes this is okay, but if you aren't sure, you should stick to qctool/bgenix for .bgen data management because of this information loss.

In [ ]:
# this step takes < 1 hour
sbatch submit_parallel_run_ukb_chr22_s20000.sh.sh

## Prepare genotype file for chromosome 1 & others, 20000 samples.

In [ ]:
# use sftp to upload to cri, take 1-2 hours on CRI
# chr1 need more than 64G memory, applying for 180G memory can be queued for some time.
cd /home/simingz/causalTWAS/ukbiobank/CRI_scripts
sftp szhao1@cri-syncmon.cri.uchicago.edu
cd /home/szhao1/causal-TWAS/data
put *

In [ ]:
qsub qsub_prep_ukbiobank_n20000_chr17_genotype.sh

In [1]:
# download plink files from CRI
sftp szhao1@cri-syncmon.cri.uchicago.edu
cd /home/szhao1/causal-TWAS/data
get ukb*

In [9]:
cd /home/simingz/causalTWAS/ukbiobank/CRI_scripts

/project2/mstephens/causalTWAS/ukbiobank/CRI_scripts


In [18]:
cmd = "cd /home/szhao1/uk-biobank/uk-biobank-genotypes; /home/szhao1/uk-biobank/uk-biobank-phenotypes/tools/plink2 --sample ukb17346_imp_chr17_v3_s487378.sample --keep /home/szhao1/causal-TWAS/data/ukbiobank_samples20000.txt --bgen ukb_imp_chr{chrom}_v3.bgen --chr {chrom} --threads 19 --make-pgen --out ~/causal-TWAS/data/ukb_chr{chrom}_s20000 --maf 0.05 --geno 0.05 --memory 30000"
qsubcmd = '''
#!/bin/bash

#PBS -N bgen2plink
#PBS -S /bin/bash
#PBS -l mem=64gb
#PBS -l walltime=48:00:00
#PBS -l nodes=1:ppn=20

cd /home/szhao1/causal-TWAS/data; bash {cmdf}
'''
masterfile = "s20000-chr2to16.sh"
mf = open(masterfile, 'w')
for chrom in [str(i) for i in range(2,17)]:
    cmdf = 'prep_ukbiobank_n20000_chr' + chrom + '_genotype.sh'
    with open(cmdf, 'w') as f:
        f.write(cmd.format(chrom=chrom))
    qsubf = 'qsub_' + cmdf
    with open(qsubf, 'w') as f2:
        f2.write(qsubcmd.format(cmdf = cmdf))
    mf.write('qsub ' + qsubf + '\n')
mf.close()

## Prepare genotype file for chromosome 1 & others, 80000 samples.

In [ ]:
# on RCC, prepare files
cd /home/simingz/causalTWAS/ukbiobank
head -n 100002 ukb17346_imp_chr17_v3_s487378.sample | tail -n 80000 | awk '{print $1"\t"$2}' > /home/simingz/causalTWAS/ukbiobank/ukbiobank_samples80000.txt

In [19]:
cmd = "cd /home/szhao1/uk-biobank/uk-biobank-genotypes; /home/szhao1/uk-biobank/uk-biobank-phenotypes/tools/plink2 --sample ukb17346_imp_chr17_v3_s487378.sample --keep /home/szhao1/causal-TWAS/data/ukbiobank_samples80000.txt --bgen ukb_imp_chr{chrom}_v3.bgen --chr {chrom} --threads 19 --make-pgen --out ~/causal-TWAS/data/ukb_chr{chrom}_s80000 --maf 0.05 --geno 0.05 --memory 30000"
qsubcmd = '''
#!/bin/bash

#PBS -N bgen2plink
#PBS -S /bin/bash
#PBS -l mem=64gb
#PBS -l walltime=48:00:00
#PBS -l nodes=1:ppn=20

cd /home/szhao1/causal-TWAS/data; bash {cmdf}
'''
masterfile = "s80000-chr2to16.sh"
mf = open(masterfile, 'w')
for chrom in [str(i) for i in range(2,17)]:
    cmdf = 'prep_ukbiobank_n80000_chr' + chrom + '_genotype.sh'
    with open(cmdf, 'w') as f:
        f.write(cmd.format(chrom=chrom))
    qsubf = 'qsub_' + cmdf
    with open(qsubf, 'w') as f2:
        f2.write(qsubcmd.format(cmdf = cmdf))
    mf.write('qsub ' + qsubf + '\n')
mf.close()

In [ ]:
# use sftp to upload to cri, take 1-2 hours on CRI
# applying for 180G memory can be queued for some time.
cd /home/simingz/causalTWAS/ukbiobank/CRI_scripts
sftp szhao1@cri-syncmon.cri.uchicago.edu
cd /home/szhao1/causal-TWAS/data
put *80000*

In [ ]:
qsub qsub_prep_ukbiobank_n80000_chr17_genotype.sh

In [ ]:
#or
bash s80000-chr2to16.sh

In [1]:
# download plink files from CRI
sftp szhao1@cri-syncmon.cri.uchicago.edu
cd /home/szhao1/causal-TWAS/data
get ukb*d

## Prepare genotype file for chromosome 1 & others, 40000 samples.

In [ ]:
# on RCC, prepare files
cd /home/simingz/causalTWAS/ukbiobank
head -n 140002 ukb17346_imp_chr17_v3_s487378.sample | tail -n 40000 | awk '{print $1"\t"$2}' > /home/simingz/causalTWAS/ukbiobank/ukbiobank_samples40000.txt

In [22]:
cmd = "cd /home/szhao1/uk-biobank/uk-biobank-genotypes; /home/szhao1/uk-biobank/uk-biobank-phenotypes/tools/plink2 --sample ukb17346_imp_chr17_v3_s487378.sample --keep /home/szhao1/causal-TWAS/data/ukbiobank_samples40000.txt --bgen ukb_imp_chr{chrom}_v3.bgen --chr {chrom} --threads 19 --make-pgen --out ~/causal-TWAS/data/ukb_chr{chrom}_s40000 --maf 0.05 --geno 0.05 --memory 30000"
qsubcmd = '''
#!/bin/bash

#PBS -N bgen2plink
#PBS -S /bin/bash
#PBS -l mem=64gb
#PBS -l walltime=48:00:00
#PBS -l nodes=1:ppn=20

cd /home/szhao1/causal-TWAS/data; bash {cmdf}
'''
masterfile = "s40000-chr1to22.sh"
mf = open(masterfile, 'w')
for chrom in [str(i) for i in range(1,23)]:
    cmdf = 'prep_ukbiobank_n40000_chr' + chrom + '_genotype.sh'
    with open(cmdf, 'w') as f:
        f.write(cmd.format(chrom=chrom))
    qsubf = 'qsub_' + cmdf
    with open(qsubf, 'w') as f2:
        f2.write(qsubcmd.format(cmdf = cmdf))
    mf.write('qsub ' + qsubf + '\n')
mf.close()

In [ ]:
# use sftp to upload to cri, take 1-2 hours on CRI
# applying for 180G memory can be queued for some time.
cd /home/simingz/causalTWAS/ukbiobank/CRI_scripts
sftp szhao1@cri-syncmon.cri.uchicago.edu
cd /home/szhao1/causal-TWAS/data
put *40000*
put ../ukbiobank_samples40000.txt

In [ ]:
bash s40000-chr1to22.sh

In [1]:
# download plink files from CRI
sftp szhao1@cri-syncmon.cri.uchicago.edu
cd /home/szhao1/causal-TWAS/data
get ukb*

# GTEx data

version 8: `/project2/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl`

version 7: need to download from here https://www.gtexportal.org/home/datasets.

## LOCUSZOOM

contains LD and gene annotation data sources for visualization

In [ ]:
cd /home/simingz/causalTWAS/locuszoom
wget https://statgen.sph.umich.edu/locuszoom/download/locuszoom_1.4.tgz